In [68]:
import fitz  # PyMuPDF
import csv
import os
import re
import pandas as pd
# Concatenar la ruta con '/data/Los-miserables.epub'
pdf_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'Los-miserables.pdf')
csv_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'Los-miserables.csv')
csv_path_clean = os.path.join(os.path.dirname(os.getcwd()), 'data', 'Los-miserables-clean.csv')
parquet_path= os.path.join(os.path.dirname(os.getcwd()), 'data', 'Los-miserables.parquet')

In [61]:
def pdf_to_csv(pdf_path, csv_path):
    # Abrir el archivo PDF
    doc = fitz.open(pdf_path)
    
    # Lista para almacenar las filas del CSV
    text_data = []
    
    # Recorrer las páginas del PDF
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # Cargar página
        text = page.get_text("text")  # Extraer texto de la página
        lines = text.split('\n')  # Dividir el texto en líneas
        
        # Procesar cada línea (dependerá del formato de tu PDF)
        for line in lines:
            # Si las columnas están separadas por espacios o comas, puedes dividir la línea
            # Aquí estamos usando split por espacio, pero puedes ajustar esto según la estructura del PDF
            split_line = line.split()  # Cambia el separador si las columnas están separadas por comas o tabuladores
            
            # Añadir la línea al texto final
            text_data.append(split_line)

    # Guardar los datos en un archivo CSV
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Escribir las filas del CSV
        writer.writerows(text_data)

    print(f"Archivo CSV guardado en: {csv_path}")

# Función para limpiar el texto (convertir a minúsculas y tratar saltos de línea como comas)
def limpiar_texto(texto):
    # Convertir todo el texto a minúsculas
    texto = texto.lower()
    
    # Usar una expresión regular para conservar solo letras (minúsculas y mayúsculas), números y saltos de línea
    # Se eliminan caracteres no deseados y los saltos de línea se reemplazan por comas
    #texto_limpio = re.sub(r'[^a-záéíóúüñ0-9\n]+', ',', texto)  # Reemplazar caracteres no deseados por espacios
    texto_limpio = re.sub(r'[^a-záéíóúüñ\n]+','',texto)  # Reemplazar caracteres no deseados por espacios
    #texto_limpio = texto_limpio.replace('\n', ',')  # Reemplazar saltos de línea por comas
    #texto_limpio = texto_limpio.replace(' ', ',')  # Reemplazar saltos de línea por comas
    #texto_limpio = texto_limpio.replace('"', ',')  # Reemplazar saltos de línea por comas
    return texto_limpio.strip()  # Eliminar espacios extra al principio y al final


# Convertir el PDF a CSV
pdf_to_csv(pdf_path, csv_path)

# Ruta del archivo CSV de entrada y salida
input_csv_path = csv_path # Ajusta la ruta según sea necesario
output_csv_path = csv_path_clean  # Ajusta la ruta según sea necesario

# Diccionario que representará el vocabulario
def tokenize(text):
    # Aquí utilizamos un simple split por espacios, pero puedes usar métodos más complejos si lo necesitas.
    return text.split(',')

def csv_clean(input_csv_path, output_csv_path):
    vocabulary = {}
    # Abrir el archivo CSV de entrada
    with open(input_csv_path, mode='r', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        
        # Leer las filas del archivo
        rows = list(reader)
    
    # Aplicar la limpieza de texto a cada celda del CSV
    for i in range(len(rows)):
        for j in range(len(rows[i])):
            # Limpiar cada celda del CSV (suponiendo que todas las celdas contienen texto)
            rows[i][j] = limpiar_texto(rows[i][j])
            if limpiar_texto(rows[i][j]) not in vocabulary:
                vocabulary[limpiar_texto(rows[i][j])] = 1
            else:
                 vocabulary[limpiar_texto(rows[i][j])] = vocabulary[limpiar_texto(rows[i][j])]+1
        # Guardar el resultado en un nuevo archivo CSV
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        
        # Escribir las filas modificadas
        writer.writerows(rows)
    
    print(f"Archivo limpio guardado en: {output_csv_path}")
    return vocabulary

vocabulary=csv_clean(input_csv_path, output_csv_path)

Archivo CSV guardado en: /Users/fau/venv-metal/code/Notebook/Vocabulary/data/Los-miserables.csv
Archivo limpio guardado en: /Users/fau/venv-metal/code/Notebook/Vocabulary/data/Los-miserables-clean.csv


In [62]:
print("El número total de palabras distintas es de:",len(vocabulary)-1, " sin considerar vacíos y manteniendo acentos")

El número total de palabras distintas es de: 13255  sin considerar vacíos y manteniendo acentos


In [63]:
# Eliminar el elemento vacío '' de la suma
sum(value for key, value in vocabulary.items() if key != '')

109276

In [64]:
# Ordenar el diccionario por frecuencia (de mayor a menor)
sorted_vocabulary_desc = sorted(vocabulary.items(), key=lambda item: item[1], reverse=True)
# Imprimir las 100 palabras más frecuentes
print("Las 100 palabras más frecuentes:")
for word, freq in sorted_vocabulary_desc[:100]:
    print(f"Palabra: {word}, Frecuencia: {freq}")

Las 100 palabras más frecuentes:
Palabra: de, Frecuencia: 5324
Palabra: la, Frecuencia: 3918
Palabra: que, Frecuencia: 3505
Palabra: y, Frecuencia: 3123
Palabra: el, Frecuencia: 3081
Palabra: en, Frecuencia: 2836
Palabra: a, Frecuencia: 2489
Palabra: se, Frecuencia: 1633
Palabra: un, Frecuencia: 1601
Palabra: no, Frecuencia: 1498
Palabra: los, Frecuencia: 1353
Palabra: una, Frecuencia: 1316
Palabra: su, Frecuencia: 1245
Palabra: por, Frecuencia: 936
Palabra: las, Frecuencia: 935
Palabra: con, Frecuencia: 924
Palabra: había, Frecuencia: 858
Palabra: del, Frecuencia: 813
Palabra: al, Frecuencia: 756
Palabra: es, Frecuencia: 749
Palabra: lo, Frecuencia: 717
Palabra: le, Frecuencia: 667
Palabra: era, Frecuencia: 650
Palabra: más, Frecuencia: 510
Palabra: como, Frecuencia: 509
Palabra: para, Frecuencia: 504
Palabra: , Frecuencia: 454
Palabra: señor, Frecuencia: 447
Palabra: pero, Frecuencia: 372
Palabra: hombre, Frecuencia: 363
Palabra: sus, Frecuencia: 344
Palabra: todo, Frecuencia: 327
Pa

In [65]:
# Ordenar el diccionario por frecuencia (de menor a mayor)
sorted_vocabulary_asc = sorted(vocabulary.items(), key=lambda item: item[1])
# Imprimir las 100 palabras menos frecuentes
print("\nLas 100 palabras menos frecuentes:")
for word, freq in sorted_vocabulary_asc[:100]:
    print(f"Palabra: {word}, Frecuencia: {freq}")


Las 100 palabras menos frecuentes:
Palabra: dominio, Frecuencia: 1
Palabra: prohibida, Frecuencia: 1
Palabra: ajenos, Frecuencia: 1
Palabra: zúrich, Frecuencia: 1
Palabra: carso, Frecuencia: 1
Palabra: ampliación, Frecuencia: 1
Palabra: granada, Frecuencia: 1
Palabra: c, Frecuencia: 1
Palabra: contactopruebatorg, Frecuencia: 1
Palabra: charlesfranoisbienvenu, Frecuencia: 1
Palabra: interesa, Frecuencia: 1
Palabra: exactos, Frecuencia: 1
Palabra: circulado, Frecuencia: 1
Palabra: ocupa, Frecuencia: 1
Palabra: reservándole, Frecuencia: 1
Palabra: consagrada, Frecuencia: 1
Palabra: galanterías, Frecuencia: 1
Palabra: sobrevino, Frecuencia: 1
Palabra: precipitáronse, Frecuencia: 1
Palabra: diezmadas, Frecuencia: 1
Palabra: perseguidas, Frecuencia: 1
Palabra: acosadas, Frecuencia: 1
Palabra: emigró, Frecuencia: 1
Palabra: trágicos, Frecuencia: 1
Palabra: espantosos, Frecuencia: 1
Palabra: emigrados, Frecuencia: 1
Palabra: germinar, Frecuencia: 1
Palabra: distracciones, Frecuencia: 1
Palabr

In [69]:
# Convertir el diccionario en un DataFrame
vocab_df = pd.DataFrame(list(vocabulary.items()), columns=['word', 'frequency'])
# Guardar el DataFrame como un archivo Parquet
vocab_df.to_parquet(parquet_path, engine='pyarrow')  # 'pyarrow' o 'fastparquet' pueden ser usados como engine

print(f"Vocabulary ha sido almacenado como Parquet file en: {parquet_path}")

Vocabulary ha sido almacenado como Parquet file en: /Users/fau/venv-metal/code/Notebook/Vocabulary/data/Los-miserables.parquet


In [75]:
df = pd.read_parquet(parquet_path, engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13256 entries, 0 to 13255
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   word       13256 non-null  object
 1   frequency  13256 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 207.2+ KB


Por:Fausto Felipe Morales
ffmogbaj@gmail.com